In [ ]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
#Build the portfolio of the user and then use MCS to forecast the portfolio performance in "years"

#Get Past 3 Year's Worth of "Bond" and "Stock" Price Data via Alpaca API Call
# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes between now and 3 years ago.(Can modify this for a more reliable forecasting)
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Set the ticker information (Choose 1 ticker for Bond and 1 ticker for stocks - index)
tickers = ["bond","stock"]

# Get 3 year's worth of historical price data for "Bond" and "Stock" (Can modify this for a more reliable forecasting)
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Display sample data
df_ticker.tail()

In [ ]:
# Reorganize the DataFrame
# Separate ticker data
bond = df_ticker[df_ticker['symbol']=='bond'].drop('symbol', axis=1)
stock = df_ticker[df_ticker['symbol']=='stock'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_ticker = pd.concat([bond, stock], axis=1, keys=["bond", "stock"])

# Display sample data
df_ticker.head()

In [ ]:
#Set the weights for bond and stock, with the option for the user to choose their risk preference
if riskpreference == "Risk Seeker":
    stockweight = ((100 - age) + 10)*0.01
    bondweight = 1 - stockweight
elif riskpreference == "Risk Neutral":
    stockweight = (100 - age)*0.01
    bondweight = 1 -stockweight
else
    stockweight = ((100-age) - 10) * 0.01
    bondweight = 1 - stockweight

In [ ]:
# Configuring a Monte Carlo simulation to forecast five years cumulative returns (or can modify to yearstoretirement = desiredretirementage - currentage)
MC_fiveyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = [bondweight,stockweight],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Printing the simulation input data
MC_fiveyear.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast five years cumulative returns
MC_fiveyear.calc_cumulative_return()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_fiveyear.plot_distribution()

# Save the plot for future usage
dist_plot.get_figure().savefig('MC_fiveyear_dist_plot.png',bbox_inches='tight')

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl = MC_fiveyear.summarize_cumulative_return()

# Print summary statistics
print(tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments in Coca-Cola and Microsoft stocks
ci_lower = round(tbl[8]*10000,2)
ci_upper = round(tbl[9]*10000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $10,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")